In [6]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [99]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# --- CONFIG ---
SEARCH_TERM = "Paracetamol"
CSV_FILENAME = "quest3plus_pmo2_data.csv"

# --- SETUP SELENIUM ---
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Enable this for headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://quest3plus.bpfk.gov.my/pmo2/index.php")

# --- WAIT AND SELECT SEARCH MODE ---
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "searchBy"))
)
select = Select(driver.find_element(By.ID, "searchBy"))
select.select_by_value("1")  # 1 = Product Name

# --- ENTER SEARCH TERM SAFELY ---
search_box = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "searchTxt"))
)
search_box.click()
search_box.clear()
search_box.send_keys(SEARCH_TERM)

# --- SUBMIT SEARCH ---
driver.find_element(By.CSS_SELECTOR, "button.btn-primary").click()

# --- WAIT FOR RESULTS TABLE ---
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#example tbody tr"))
)

time.sleep(2)  # Let JS fully render results
driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ED80E3+60707]
	GetHandleVerifier [0x00ED8124+60772]
	(No symbol) [0x00D00683]
	(No symbol) [0x00D48660]
	(No symbol) [0x00D489FB]
	(No symbol) [0x00D91022]
	(No symbol) [0x00D6D094]
	(No symbol) [0x00D8E824]
	(No symbol) [0x00D6CE46]
	(No symbol) [0x00D3C5D3]
	(No symbol) [0x00D3D424]
	GetHandleVerifier [0x0111BBC3+2435075]
	GetHandleVerifier [0x01117163+2416035]
	GetHandleVerifier [0x0113350C+2531660]
	GetHandleVerifier [0x00EEF1B5+155125]
	GetHandleVerifier [0x00EF5B5D+182173]
	GetHandleVerifier [0x00EDF9B8+91640]
	GetHandleVerifier [0x00EDFB60+92064]
	GetHandleVerifier [0x00ECA620+4704]
	BaseThreadInitThunk [0x75585D49+25]
	RtlInitializeExceptionChain [0x7730CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x7730CDC1+561]


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# --- CONFIG ---
SEARCH_TERM = "PARACAP PARACETAMOL 500MG TABLET"
CSV_FILENAME = "quest3plus_pmo2_data.csv"

# --- SETUP SELENIUM ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Enable for background scraping
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://quest3plus.bpfk.gov.my/pmo2/index.php")

# --- WAIT AND SELECT SEARCH MODE ---
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "searchBy")))
select = Select(driver.find_element(By.ID, "searchBy"))
select.select_by_value("1")  # 1 = Product Name

# --- ENTER SEARCH TERM SAFELY ---
search_box = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "searchTxt"))
)
search_box.click()
search_box.clear()
search_box.send_keys(SEARCH_TERM)

# --- SUBMIT SEARCH ---
driver.find_element(By.CSS_SELECTOR, "button.btn-primary").click()

# --- WAIT FOR RESULTS TABLE ---
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#example tbody tr"))
)
time.sleep(2)

# --- SCRAPE RESULTS ---
rows = driver.find_elements(By.CSS_SELECTOR, "#searchTable tbody tr")
results = []

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) >= 4:
        # Updated order based on new structure
        reg_no = cols[1].text.strip()
        product_name = cols[2].text.strip()
        holder = cols[3].text.strip()

        # Click the <a> element to trigger detail modal
        try:
            detail_link = cols[1].find_element(By.TAG_NAME, "a")  # second column = reg_no
            detail_link.click()
            time.sleep(2)
        except:
            print(f"❌ Could not click detail link for {reg_no}")
            continue

        # Extract detail from modal
        try:
            detail_box = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "product_detail"))
            )
            detail_text = detail_box.text.strip()
        except:
            detail_text = "No detail found"

        results.append([SEARCH_TERM, reg_no, product_name, holder, detail_text])
        print(f"✅ {product_name} | {reg_no}\n📄 Detail: {detail_text[:100]}...\n")

driver.quit()

# --- SAVE TO CSV ---
with open(CSV_FILENAME, "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Search Term", "Product Name", "Registration Number", "Active Ingredient", "Detail Info"])
    writer.writerows(results)

print(f"\n✅ Scraping complete. Data saved to: {CSV_FILENAME}")


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00FC80E3+60707]
	GetHandleVerifier [0x00FC8124+60772]
	(No symbol) [0x00DF0683]
	(No symbol) [0x00E38660]
	(No symbol) [0x00E389FB]
	(No symbol) [0x00E81022]
	(No symbol) [0x00E5D094]
	(No symbol) [0x00E7E824]
	(No symbol) [0x00E5CE46]
	(No symbol) [0x00E2C5D3]
	(No symbol) [0x00E2D424]
	GetHandleVerifier [0x0120BBC3+2435075]
	GetHandleVerifier [0x01207163+2416035]
	GetHandleVerifier [0x0122350C+2531660]
	GetHandleVerifier [0x00FDF1B5+155125]
	GetHandleVerifier [0x00FE5B5D+182173]
	GetHandleVerifier [0x00FCF9B8+91640]
	GetHandleVerifier [0x00FCFB60+92064]
	GetHandleVerifier [0x00FBA620+4704]
	BaseThreadInitThunk [0x753D5D49+25]
	RtlInitializeExceptionChain [0x7739CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x7739CE91+561]
